In [2]:
import torch
from model.model import parsingNet
from torch.onnx import export
import onnx
import onnxruntime
import numpy as np
import scipy
import cv2
from data.constant import culane_row_anchor

In [3]:
net = parsingNet(pretrained = False, backbone='18',cls_dim = (201,18,4),use_aux=False) # we dont need auxiliary segmentation in testing

state_dict = torch.load('checkpoints/culane_18.pth', map_location='cpu')['model']
compatible_state_dict = {}
for k, v in state_dict.items():
  if 'module.' in k:
    compatible_state_dict[k[7:]] = v
  else:
    compatible_state_dict[k] = v

net.load_state_dict(compatible_state_dict, strict=False)
# 设置模型为推理模式
net.eval()

x = torch.randn(1, 3, 288, 800)
torch_out = net(x)

d:\anaconda3\envs\deeplane\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\anaconda3\envs\deeplane\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
export(net, x, 'culane_18.onnx', verbose=True, input_names=['input'], output_names=['output'], opset_version=11)

In [5]:
onnx_model = onnx.load('culane_18.onnx')

In [6]:
# onnx_model = onnx.load("test.onnx")
onnx.checker.check_model(onnx_model)

ort_session = onnxruntime.InferenceSession("culane_18.onnx")
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)
print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [7]:
from PIL import Image
import torchvision.transforms as transforms

# img_path = 'I:/Ultra-Fast-Lane-Detection/CULane/driver_37_30frame/05181432_0203.MP4/04320.jpg'
img_path = 'D:/Projects/deeplearning/data/CULane/driver_23_30frame/05151640_0419.MP4/00000.jpg'

img = Image.open(img_path)
# print(type(img))
# image = np.asarray(img, dtype=np.float32)
# image = np.transpose(image, (2, 0, 1))

img_transforms = transforms.Compose([
        transforms.Resize((288, 800)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])


image = img_transforms(img).unsqueeze(0)
print(image.shape)

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image)}
ort_outs = ort_session.run(None, ort_inputs)

ort_outs = ort_outs[0][0]
# print(ort_outs)

print(ort_outs.shape)

img_w, img_h = 1640, 590
row_anchor = culane_row_anchor
cls_num_per_lane = 18

col_sample = np.linspace(0, 800 - 1, 200)
col_sample_w = col_sample[1] - col_sample[0]

ort_outs = ort_outs[:, ::-1, :]
prob = scipy.special.softmax(ort_outs[:-1, :, :], axis=0)
idx = np.arange(200) + 1
idx = idx.reshape(-1, 1, 1)
loc = np.sum(prob * idx, axis=0)
out_j = np.argmax(ort_outs, axis=0)
loc[out_j == 200] = 0
out_j = loc

# import pdb; pdb.set_trace()
vis = cv2.imread(img_path)
for i in range(out_j.shape[1]):
    if np.sum(out_j[:, i] != 0) > 2:
        for k in range(out_j.shape[0]):
            if out_j[k, i] > 0:
                ppp = (int(out_j[k, i] * col_sample_w * img_w / 800) - 1, int(img_h * (row_anchor[cls_num_per_lane-1-k]/288)) - 1 )
                cv2.circle(vis,ppp,5,(0,255,0),-1)
cv2.imwrite('result.jpg', vis)

torch.Size([1, 3, 288, 800])
(201, 18, 4)


True

In [2]:
import os
import tensorrt as trt

In [3]:
TRT_LOGGER = trt.Logger()
onnx_file_path = 'culane_18.onnx'
engine_file_path = 'culane_18.engine'

BATCH_SIZE = 1
IN_H = 288
IN_W = 800

In [4]:
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(EXPLICIT_BATCH)
parser = trt.OnnxParser(network, TRT_LOGGER)
# parse onnx
if isinstance(onnx_file_path, str):
  parse_valid = parser.parse_from_file(onnx_file_path)
elif isinstance(onnx_file_path, onnx.ModelProto):
  parse_valid = parser.parse(onnx_file_path.SerializeToString())
else:
  raise TypeError('Unsupported onnx model type!')

if not parse_valid:
  error_msgs = ''
  for error in range(parser.num_errors):
    error_msgs += f'{parser.get_error(error)}\n'
  raise RuntimeError(f'Failed to parse onnx, {error_msgs}')

# config builder
config = builder.create_builder_config()
config.max_workspace_size = 1 << 30

profile = builder.create_optimization_profile()
profile.set_shape_input('input', *[[BATCH_SIZE, 3, IN_H, IN_W]]*3)
if config.add_optimization_profile(profile) < 0:
  TRT_LOGGER.warning(f'Invalid optimization profile {profile}.')

# config.set_flag(trt.BuilderFlag.FP16)

engine = builder.build_engine(network, config)
assert engine is not None, 'Failed to create TensorRT engine'
# save(engine, output_file_prefix + '.engine')
with open(engine_file_path, mode='wb') as f:
  if isinstance(engine, trt.ICudaEngine):
    engine = engine.serialize()
  f.write(bytearray(engine))

C:\Users\claud\AppData\Local\Temp\ipykernel_22372\1327851646.py:22: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 30
C:\Users\claud\AppData\Local\Temp\ipykernel_22372\1327851646.py:25: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config)


In [ ]:
ENGINE_PATH='/content/drive/MyDrive/model/culane_18.engine'
trt_logger = trt.Logger(trt.Logger.INFO)
runtime = trt.Runtime(trt_logger)
with open(ENGINE_PATH, "rb") as f:
  engine = runtime.deserialize_cuda_engine(f.read())

In [ ]:
context = engine.create_execution_context()

INPUT_DATA_TYPE = np.float32
stream = cuda.Stream()
host_in = cuda.pagelocked_empty(trt.volume(engine.get_binding_shape(0)), dtype=INPUT_DATA_TYPE)
host_out = cuda.pagelocked_empty(trt.volume(engine.get_binding_shape(1)), dtype=INPUT_DATA_TYPE)

device_in = cuda.mem_alloc(host_in.nbytes)
device_out = cuda.mem_alloc(host_out.nbytes)
bindings = [int(device_in), int(device_out)]

In [ ]:
row_anchor = culane_row_anchor
cls_num_per_lane = 18
col_sample = np.linspace(0, 800 - 1, 200)
col_sample_w = col_sample[1] - col_sample[0]

img_transforms = transforms.Compose([
  transforms.Resize((288, 800)),
  transforms.ToTensor(),
  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

def to_numpy(tensor):
  return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [ ]:
image_path = '/content/drive/MyDrive/data/test_img/05220.jpg'

img = Image.open(image_path)
img = img_transforms(img).unsqueeze(0)
np.copyto(host_in, img.ravel())
cuda.memcpy_htod_async(device_in, host_in, stream)
context.execute_async_v2(bindings, stream.handle)
cuda.memcpy_dtoh_async(host_out, device_out, stream)
stream.synchronize()
ort_outs = host_out.reshape(201,18,4)
ort_outs = ort_outs[:, ::-1, :]
prob = scipy.special.softmax(ort_outs[:-1, :, :], axis=0)
idx = np.arange(200) + 1
idx = idx.reshape(-1, 1, 1)
loc = np.sum(prob * idx, axis=0)
out_j = np.argmax(ort_outs, axis=0)
loc[out_j == 200] = 0
out_j = loc

image = cv2.imread(image_path)
img_h, img_w = image.shape[:2]
for i in range(out_j.shape[1]):
  if np.sum(out_j[:, i] != 0) > 2:
    for k in range(out_j.shape[0]):
      if out_j[k, i] > 0:
        ppp = (int(out_j[k, i] * col_sample_w * img_w / 800) - 1, int(img_h * (row_anchor[cls_num_per_lane-1-k]/288)) - 1 )
        cv2.circle(image,ppp,5,(0,255,0),-1)
cv2.imwrite('/content/result.jpg', image)